# Webscraping Instagram

## Connection use Selenium

In [216]:
#import libraries
from selenium import webdriver
import urllib.request
import pandas as pd
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import re

In [2]:
# Prepare the code for easily opening new browser window
driver_option = webdriver.ChromeOptions()
driver_option.add_argument(" — incognito")
chromedriver_path = '/home/igor/Estudos/Ciencia de Dados e Big Data/RI/chromedriver_linux'

def create_webdriver():
    return webdriver.Chrome(executable_path=chromedriver_path, options = driver_option)

In [42]:
#Open Google Chrome
driver = create_webdriver()
#Open page instagood
driver.get("https://www.instagram.com/explore/tags/instagood/")
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]')))
photo = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]')
ActionChains(driver).click(photo).perform()

In [ ]:
#Capture URLs
instagram_url = []
for i in range(30000):
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, "//a[@class='HBoOv coreSpriteRightPaginationArrow']")))
        instagram_url.append(driver.current_url)
    except:
        ActionChains(driver).send_keys(Keys.ARROW_LEFT).perform()
    ActionChains(driver).send_keys(Keys.ARROW_RIGHT).perform()

In [248]:
len(instagram_url)

16028

In [100]:
#Save URLs
pd_insta_url = pd.DataFrame(instagram_url, columns= ['urls'])
pd_insta_url.to_csv('instagram_urls.csv')

In [228]:
pd_insta =  pd.read_csv('instagram_urls.csv', index_col = 0)
pd_insta['user'] = ''
pd_insta['Description'] = ''
pd_insta['Likes'] = 0
pd_insta['status'] = ''
len(pd_insta)

16028

In [208]:
#Open Google Chrome
driver = create_webdriver()

In [ ]:
wait = WebDriverWait(driver, 10)
user_xpath = "//h2[@class='BrX75']/a"
image_description_xpath = "//div[@class='KL4Bh']/img"
notfound = "//div[@class='error-container -cx-PRIVATE-ErrorPage__errorContainer -cx-PRIVATE-ErrorPage__errorContainer__']"
resuts = []
for i in range(len(pd_insta)):
    driver.get(pd_insta.iloc[i,0])
    try:
        driver.find_element_by_xpath(notfound)
        pd_insta.iloc[i,1:] = ['not found', 'not found', 0, 'not found']
    except:
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, user_xpath)))
            user = driver.find_element_by_xpath(user_xpath).text
            wait.until(EC.presence_of_element_located((By.XPATH, image_description_xpath)))
            image = driver.find_element_by_xpath(image_description_xpath).get_attribute('alt')
            image = image.replace('Image may contain: ','')
            likes = driver.find_element_by_xpath("//a[@class='zV_Nj']").text
            likes = int(re.sub('\D','',likes))
            pd_insta.iloc[i,1:] = [user, image, likes, 'ok']
        except:
            print('timeout')

In [239]:
pd_insta.head()

,urls,user,Description,Likes,status
0,https://www.instagram.com/p/BzdLqZUIL5a/,world.traveling.guide,"outdoor, water and nature",1723,ok
1,https://www.instagram.com/p/BzdLwXQpUEb/,seungcheol__j,one or more people and closeup,17001,ok
2,https://www.instagram.com/p/BzdMq_8F5ex/,robert.beczarski,one or more people and closeup,790,ok
3,https://www.instagram.com/p/BzdM0kZhUt9/,___.a.07,one or more people and closeup,1318,ok
4,https://www.instagram.com/p/BzdNBPald1T/,ch_1_eeee,No photo description available.,293,ok


In [238]:
pd_insta.groupby('status').count()

,urls,user,Description,Likes
status,,,,
,665,665,665,665
not found,903,903,903,903
ok,14460,14460,14460,14460


In [247]:
filter = pd_insta['status'].str.contains('ok')
pd_insta_filled = pd_insta[filter].iloc[:,:-1]
pd_insta_filled.head()

,urls,user,Description,Likes
0,https://www.instagram.com/p/BzdLqZUIL5a/,world.traveling.guide,"outdoor, water and nature",1723
1,https://www.instagram.com/p/BzdLwXQpUEb/,seungcheol__j,one or more people and closeup,17001
2,https://www.instagram.com/p/BzdMq_8F5ex/,robert.beczarski,one or more people and closeup,790
3,https://www.instagram.com/p/BzdM0kZhUt9/,___.a.07,one or more people and closeup,1318
4,https://www.instagram.com/p/BzdNBPald1T/,ch_1_eeee,No photo description available.,293


In [266]:
pd_insta_filled['urls'] = pd_insta_filled['urls'].str.replace('https://www.instagram.com/p/','')
pd_insta_filled['urls'] = pd_insta_filled['urls'].str.replace('/$','')
pd_insta_filled.rename(columns= {'urls' : 'id_post'}, inplace=True)
pd_insta_filled.head(10)

,id_post,user,Description,Likes
0,BzdLqZUIL5a,world.traveling.guide,"outdoor, water and nature",1723
1,BzdLwXQpUEb,seungcheol__j,one or more people and closeup,17001
2,BzdMq_8F5ex,robert.beczarski,one or more people and closeup,790
3,BzdM0kZhUt9,___.a.07,one or more people and closeup,1318
4,BzdNBPald1T,ch_1_eeee,No photo description available.,293
5,BzdMqlkFvJk,marcelaferrato_store,No photo description available.,3540
6,BzdLcjVJjAS,melo_theecat,cat,2096
7,BzdM9FbiL1x,chris.boom.official,one or more people,3261
8,BzdOw-0BevB,monicaardaleysia_16,one or more people and closeup,3364
9,BzdT0pVBeTu,kreationsbytishpartyandevents,No photo description available.,5


In [271]:
no_desc = ~pd_insta_filled['Description'].str.contains('No photo description available.')
pd_insta_filled = pd_insta_filled[no_desc]
pd_insta_filled.head(10)

,id_post,user,Description,Likes
0,BzdLqZUIL5a,world.traveling.guide,"outdoor, water and nature",1723
1,BzdLwXQpUEb,seungcheol__j,one or more people and closeup,17001
2,BzdMq_8F5ex,robert.beczarski,one or more people and closeup,790
3,BzdM0kZhUt9,___.a.07,one or more people and closeup,1318
6,BzdLcjVJjAS,melo_theecat,cat,2096
7,BzdM9FbiL1x,chris.boom.official,one or more people,3261
8,BzdOw-0BevB,monicaardaleysia_16,one or more people and closeup,3364
10,BzdT0dqoa-e,magdalena_blaszczak,one or more people and closeup,20
12,BzdT0DvnjkM,ugotmail56,dog and eyeglasses,21
15,BzdTzxFnPoG,flow617sc,shoes,19


In [ ]:
pd_insta_filled.to_csv('instagood_photos.csv', index= False)

#### Check these
https://www.linkedin.com/pulse/scraping-instagram-selenuim-aman-khalid/

https://selenium-python.readthedocs.io/api.html#module-selenium.webdriver.common.action_chains

In [21]:
#Close connection
#driver.quit()